### Introduction 

### Setup

In [1]:
import pymongo
import pandas as pd
from pandas.io.json import json_normalize 
from pymongo import MongoClient
import numpy as np 

In [2]:
 #point the client at mongo URI 
client = MongoClient('mongodb://localhost:27017')
#select database
db = client['tipster']

Two mongodb collections - one for results meeting-horses-place , one for form - horse

In [3]:
form_collection = db.form
data = form_collection.find({},{"_id":0}) 
form_df = pd.DataFrame(json_normalize(data, 'form',['horse_name']))

In [4]:
# flatten data and put into dataframe 
meetings_collection = db.meetings
data=meetings_collection.find({},{"_id":0})
meeting_runners_df= pd.DataFrame(json_normalize(data,'runners',['meeting_url','meeting_date_time']))
meeting_runners_df['won'] = np.where(meeting_runners_df['place'] == 1.0, 1,0 )

for each horse-date check if winner in any of the previous races  

for each horse get the data of the first win  
for each horse-date horse is a winner if date (strictly) > first win 

In [5]:
winners = meeting_runners_df[meeting_runners_df.won == 1][['horse_name','meeting_date_time']]
winners.rename(columns={"meeting_date_time":"win_date"},inplace=True)
winners = winners.groupby(["horse_name"]).min().reset_index() 
X = pd.merge(meeting_runners_df,winners,on="horse_name",how="left")
X["has_winning_form"] = np.where((X.meeting_date_time > X.win_date),1,0)

#### Form against competitors
Generate pairs of horse in every race- cartesian product of horses for each meeting -  
also need meeting date time so we can just count results previous to meeting when we do merge  

'Winner' is horse that is better placed then other horse - 'loser'

if date of win is prior to meeting date update beaten other horse in race in 'x' to true

In [6]:
mdf = meeting_runners_df
mdfl = pd.DataFrame(mdf[['meeting_url','horse_name','place','meeting_date_time']])
mdfr = mdfl
competitors = mdfl.merge(mdfr, on='meeting_url')

#get name of winner and loser 
competitors['winner'] = np.where(competitors.place_x < competitors.place_y, 
                                 competitors.horse_name_x, competitors.horse_name_y) 
competitors['loser'] = np.where(competitors.place_x >= competitors.place_y, competitors.horse_name_x, competitors.horse_name_y)
#remove duplicate pairs
competitors = competitors[competitors['loser'] != competitors['winner']]
#rename columns for clarity
competitors.rename(columns={"place_x":"winner_place","place_y":"loser_place"},inplace=True)  

#### collateral form   
##### if A beat B and B beat C in another race , then A has collateral form against C
if A and C are competitors in the same race 
if A is a winner and B is a loser in the most race they ran togther, and B won against C in the most recent race they ran together before that, then A has collateral form against C. 



if B and C are competitors in the same race 
if B lost against A in the most previous race they ran together and C lost against A in the most previous race they ran together, and B was better placed than C then A has collateral form against C.

##### if A beat B in one meeting and A beat C in another meeting and B had a better place than C against A, then B has indirect form against C 



In [7]:
#join to get horse A,B and C  
collateral_form = competitors.merge(competitors, left_on="loser", right_on="winner")
#don't include three horses in same race
collateral_form = collateral_form[collateral_form['meeting_url_x']!=collateral_form['meeting_url_y']]

In [8]:
# sort for merge_asof
cf1 = collateral_form[['winner_x','loser_y','meeting_date_time_y_y']].sort_values(by='meeting_date_time_y_y')
cm1 = competitors[['horse_name_x','horse_name_y','meeting_date_time_y']].sort_values(by='meeting_date_time_y')

#find pairs of horses in same race and see if one has collateral 
res = pd.merge_asof(cm1,cf1,allow_exact_matches=False, \
                    left_by=['horse_name_x','horse_name_y'], right_by=['winner_x','loser_y'], \
                    left_on='meeting_date_time_y', right_on='meeting_date_time_y_y')
#outer join so remove nulls
res1 = res[res['winner_x'].notnull()]

In [11]:
mrd = meeting_runners_df
has_cf = pd.merge(res1, mrd, left_on=['winner_x','meeting_date_time_y'],right_on=['horse_name','meeting_date_time'],how='right')


horse may have beaten a horse that beat more than one  horse in the same race

In [12]:
p = has_cf[['horse_name', 'meeting_url','winner_x','loser_y','meeting_date_time_y_y']].\
    groupby(['horse_name', 'meeting_url']).meeting_date_time_y_y.agg("max").reset_index()

p['has_cf'] = np.where(p.meeting_date_time_y_y.isnull(),0,1)
X['has_cf'] = p['has_cf']
X[X.has_cf == 0]

,age,horse_name,horse_url,last_ran,place,meeting_url,meeting_date_time,won,win_date,has_winning_form,has_cf
0,7yo Geld,Green Zone,/racing/form-profile-2020075,1153 days,1.0,https://gg.co.uk/racing/22-dec-2018/newcastle-...,2018-12-22 11:40:00,1,2018-09-26 17:20:00,1,0
1,10yo Geld,Copt Hill,/racing/form-profile-1374915,3006 days,2.0,https://gg.co.uk/racing/22-dec-2018/newcastle-...,2018-12-22 11:40:00,0,2018-12-13 14:20:00,1,0
2,8yo Geld,Fiosrach,/racing/form-profile-2010345,7 days,3.0,https://gg.co.uk/racing/22-dec-2018/newcastle-...,2018-12-22 11:40:00,0,NaT,0,0
3,6yo Mare,Lady Samback,/racing/form-profile-2408905,107 days,4.0,https://gg.co.uk/racing/22-dec-2018/newcastle-...,2018-12-22 11:40:00,0,NaT,0,0
5,7yo Geld,Bingo Dʼolivate,/racing/form-profile-2316805,23 days,6.0,https://gg.co.uk/racing/22-dec-2018/newcastle-...,2018-12-22 11:40:00,0,2018-11-29 15:00:00,1,0
10,7yo Geld,Solway Lark,/racing/form-profile-2300505,17 days,11.0,https://gg.co.uk/racing/22-dec-2018/newcastle-...,2018-12-22 11:40:00,0,NaT,0,0
12,7yo Geld,Glance Back,/racing/form-profile-2130455,45 days,1.0,https://gg.co.uk/racing/14-dec-2018/bangor-on-...,2018-12-14 11:50:00,1,2018-12-14 11:50:00,0,0
13,7yo Geld,Lickpenny Larry,/racing/form-profile-2330575,243 days,2.0,https://gg.co.uk/racing/14-dec-2018/bangor-on-...,2018-12-14 11:50:00,0,NaT,0,0
14,8yo Geld,Roxyfet,/racing/form-profile-2130825,2088 days,3.0,https://gg.co.uk/racing/14-dec-2018/bangor-on-...,2018-12-14 11:50:00,0,NaT,0,0
15,5yo Mare,Miss Amelia,/racing/form-profile-2483735,23 days,4.0,https://gg.co.uk/racing/14-dec-2018/bangor-on-...,2018-12-14 11:50:00,0,NaT,0,0


### Data|

### Features


which horses in a meeting are winners as of meeting date?  

has a horse in a meeting beaten another horse in the same meeting?

for a given pair of horses which one did better against a third horse?  
    how similar were the conditions? 

time since last race (fitness and value of form)

does the horse need to improve - is it a champion

will form improve?   
margin of victory - ( bigger implies form won't change )  (low fitness + ease => improving form) 

commentary:  
   activity during race  
   finish  
   speed 

is progressively better

above or below average time for course?

raised or lowered in class?

proven at distance 

proven for going

weight

draw - look up advantage of draw in course

type of race - handicap/flat/jumps

### Training


In [ ]:
import sklearn
from sklearn.linear_model import LogisticRegression
# from sklearn.cross_validation import train_test_split 



### Predict